# Text Mining for ESG Score Analysis & Prediction : Using News data

#Import libraries

In [57]:
import sklearn
import random
import numpy as np
import pandas as pd
import tqdm
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image #image 불러오기
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor

np.random.seed(42)
%matplotlib inline

In [58]:
#access Google drive file
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [59]:
#한글 폰트 적용
try:
  mpl.rc('font', family='NanumBarunGothic')
  sns.set(font="NanumBarunGothic", 
        rc={"axes.unicode_minus":False},
        style='darkgrid')
except:
  !sudo apt-get install -y fonts-nanum
  !sudo fc-cache -fv
  !rm ~/.cache/matplotlib -rf
  mpl.rc('font', family='NanumBarunGothic')
  sns.set(font="NanumBarunGothic", 
        rc={"axes.unicode_minus":False},
        style='darkgrid')

In [119]:
# 그림 저장
image_path = '/content/drive/MyDrive/LIS3821/images/'

def save_fig(fig_name, tight_layout=True, fig_extension="png", resolution=300):
    #해상도 dpi=300
    path = image_path + fig_name + "." + fig_extension
    print("그림 저장: ", fig_name)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Preprocessing

## Data Integration

In [154]:
#Unnamed: 0 열 drop
df_ESG_2021 = pd.read_csv('/content/drive/MyDrive/LIS3821/datasets/KCGS_ESG등급_2021.csv').drop(columns=['Unnamed: 0', '평가연도'], axis=1)
df_ESG_2021.head()

,기업명,ESG 종합등급,환경,사회,지배구조
0,AJ네트웍스,B,D,B,B
1,AK홀딩스,B+,B,B+,B+
2,BGF,A,A,A+,A
3,BGF리테일,A,A,A+,A
4,BNK금융지주,A+,A,A+,A+


In [155]:
#Unnamed: 0 열 drop
df_ESG_2020 = pd.read_csv('/content/drive/MyDrive/LIS3821/datasets/KCGS_ESG등급_2020.csv').drop(columns=['Unnamed: 0', '평가연도'], axis=1)
df_ESG_2020.head(10)

,기업명,ESG 종합등급,환경,사회,지배구조
0,AJ네트웍스,B,D,B,B
1,AK홀딩스,B+,C,B+,B+
2,BGF,B+,B,A,B+
3,BGF리테일,A,B+,B+,A
4,BNK금융지주,A+,B+,A+,A+
5,BYC,B,D,B,B+
6,CJ,A,B,A,A
7,CJ CGV,A,B,A,A
8,CJ대한통운,A,A,B+,A
9,CJ씨푸드,B,C,B+,B+


In [156]:
#Unnamed: 0 열 drop
df_ESG_2019 = pd.read_csv('/content/drive/MyDrive/LIS3821/datasets/KCGS_ESG등급_2019.csv').drop(columns=['Unnamed: 0', '평가연도'], axis=1)
df_ESG_2019.head()

,기업명,ESG 종합등급,환경,사회,지배구조
0,AJ네트웍스,B,D,B,B
1,AK홀딩스,B,C,B,B
2,BGF,B,C,A,B
3,BGF리테일,B,C,B+,B+
4,BNK금융지주,A+,B+,A+,A+


In [157]:
df_ESG_all = [df_ESG_2021, df_ESG_2020, df_ESG_2019]

In [158]:
for df in df_ESG_all:
  #열 이름 띄어쓰기 없애기
  df.columns = ['기업명', '종합등급', '환경', '사회', '지배구조']

In [216]:
df_2021 = df_ESG_2021.copy()
df_2020 = df_ESG_2020.copy()
df_2019 = df_ESG_2019.copy()

In [217]:
df_m = pd.merge(df_2021, df_2020, on='기업명', how='inner', suffixes=('_2021', '_2020'))
df_m.head()

,기업명,종합등급_2021,환경_2021,사회_2021,지배구조_2021,종합등급_2020,환경_2020,사회_2020,지배구조_2020
0,AJ네트웍스,B,D,B,B,B,D,B,B
1,AK홀딩스,B+,B,B+,B+,B+,C,B+,B+
2,BGF,A,A,A+,A,B+,B,A,B+
3,BGF리테일,A,A,A+,A,A,B+,B+,A
4,BNK금융지주,A+,A,A+,A+,A+,B+,A+,A+


In [218]:
df_2019.rename(columns={'종합등급':'종합등급_2019', '환경': '환경_2019', '사회':'사회_2019', '지배구조':'지배구조_2019'}, inplace=True)
df_2019.head()

,기업명,종합등급_2019,환경_2019,사회_2019,지배구조_2019
0,AJ네트웍스,B,D,B,B
1,AK홀딩스,B,C,B,B
2,BGF,B,C,A,B
3,BGF리테일,B,C,B+,B+
4,BNK금융지주,A+,B+,A+,A+


In [219]:
df_ESG = df_m.merge(df_2019, on = '기업명', how='inner')
df_ESG.head()

,기업명,종합등급_2021,환경_2021,사회_2021,지배구조_2021,종합등급_2020,환경_2020,사회_2020,지배구조_2020,종합등급_2019,환경_2019,사회_2019,지배구조_2019
0,AJ네트웍스,B,D,B,B,B,D,B,B,B,D,B,B
1,AK홀딩스,B+,B,B+,B+,B+,C,B+,B+,B,C,B,B
2,BGF,A,A,A+,A,B+,B,A,B+,B,C,A,B
3,BGF리테일,A,A,A+,A,A,B+,B+,A,B,C,B+,B+
4,BNK금융지주,A+,A,A+,A+,A+,B+,A+,A+,A+,B+,A+,A+


## Cleaning Missing Values

In [220]:
df_ESG.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 775 entries, 0 to 774
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   기업명        775 non-null    object
 1   종합등급_2021  775 non-null    object
 2   환경_2021    775 non-null    object
 3   사회_2021    775 non-null    object
 4   지배구조_2021  775 non-null    object
 5   종합등급_2020  775 non-null    object
 6   환경_2020    775 non-null    object
 7   사회_2020    775 non-null    object
 8   지배구조_2020  775 non-null    object
 9   종합등급_2019  775 non-null    object
 10  환경_2019    775 non-null    object
 11  사회_2019    775 non-null    object
 12  지배구조_2019  775 non-null    object
dtypes: object(13)
memory usage: 84.8+ KB


In [221]:
year1 = '_2021'
year2 = '_2020'
year3 = '_2019'
condition1 = (df_ESG[f'종합등급{year1}'] == '-') | (df_ESG[f'환경{year1}'] == '-') | (df_ESG[f'사회{year1}'] == '-') | (df_ESG[f'지배구조{year1}'] == '-')
condition2 = (df_ESG[f'종합등급{year2}'] == '-') | (df_ESG[f'환경{year2}'] == '-') | (df_ESG[f'사회{year2}'] == '-') | (df_ESG[f'지배구조{year2}'] == '-')
condition3 = (df_ESG[f'종합등급{year3}'] == '-') | (df_ESG[f'환경{year3}'] == '-') | (df_ESG[f'사회{year3}'] == '-') | (df_ESG[f'지배구조{year3}'] == '-')

df_ESG[condition1 | condition2 | condition3]

,기업명,종합등급_2021,환경_2021,사회_2021,지배구조_2021,종합등급_2020,환경_2020,사회_2020,지배구조_2020,종합등급_2019,환경_2019,사회_2019,지배구조_2019
23,F&F,-,-,-,-,-,-,-,-,-,-,-,-
33,HDC현대산업개발,B,B,C,B+,B,C,B,B+,-,-,-,-
61,LG에너지솔루션,-,-,-,-,-,-,-,-,-,-,-,-
102,SK바이오사이언스,-,-,-,-,-,-,-,-,-,-,-,-
103,SK바이오사이언스,-,-,-,-,-,-,-,-,-,-,-,-
104,SK스퀘어,-,-,-,-,-,-,-,-,-,-,-,-
105,SK스퀘어,-,-,-,-,-,-,-,-,-,-,-,-
106,SK아이이테크놀로지,-,-,-,-,-,-,-,-,-,-,-,-
107,SK아이이테크놀로지,-,-,-,-,-,-,-,-,-,-,-,-
111,STX중공업,B,C,B,B+,B,C,B+,B+,-,-,-,-


In [222]:
#'-'값 NaN으로 변경
df_ESG.replace('-', np.NaN, inplace=True)
df_ESG.isna().sum()

기업명           0
종합등급_2021    12
환경_2021      12
사회_2021      12
지배구조_2021    12
종합등급_2020    22
환경_2020      22
사회_2020      22
지배구조_2020    22
종합등급_2019    41
환경_2019      41
사회_2019      41
지배구조_2019    41
dtype: int64

In [223]:
#결측값(missing value)있는 행 모두 제거
df_ESG = df_ESG.dropna(axis=0)
df_ESG.isna().sum()

기업명          0
종합등급_2021    0
환경_2021      0
사회_2021      0
지배구조_2021    0
종합등급_2020    0
환경_2020      0
사회_2020      0
지배구조_2020    0
종합등급_2019    0
환경_2019      0
사회_2019      0
지배구조_2019    0
dtype: int64

In [224]:
df_ESG.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 734 entries, 0 to 774
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   기업명        734 non-null    object
 1   종합등급_2021  734 non-null    object
 2   환경_2021    734 non-null    object
 3   사회_2021    734 non-null    object
 4   지배구조_2021  734 non-null    object
 5   종합등급_2020  734 non-null    object
 6   환경_2020    734 non-null    object
 7   사회_2020    734 non-null    object
 8   지배구조_2020  734 non-null    object
 9   종합등급_2019  734 non-null    object
 10  환경_2019    734 non-null    object
 11  사회_2019    734 non-null    object
 12  지배구조_2019  734 non-null    object
dtypes: object(13)
memory usage: 80.3+ KB


In [225]:
#csv파일 만들기
df_ESG.to_csv('/content/drive/MyDrive/LIS3821/datasets/KCGS_ESG등급_2021-2019.csv')

## Ordinal Encoding

In [226]:
df_ESG.head()

,기업명,종합등급_2021,환경_2021,사회_2021,지배구조_2021,종합등급_2020,환경_2020,사회_2020,지배구조_2020,종합등급_2019,환경_2019,사회_2019,지배구조_2019
0,AJ네트웍스,B,D,B,B,B,D,B,B,B,D,B,B
1,AK홀딩스,B+,B,B+,B+,B+,C,B+,B+,B,C,B,B
2,BGF,A,A,A+,A,B+,B,A,B+,B,C,A,B
3,BGF리테일,A,A,A+,A,A,B+,B+,A,B,C,B+,B+
4,BNK금융지주,A+,A,A+,A+,A+,B+,A+,A+,A+,B+,A+,A+


In [227]:
#등급 고유값 찾기
rank_nested_list = df_ESG[df_ESG.columns.difference(['기업명'])].values.tolist() #nested list
rank_list = sum(rank_nested_list, [])
rank_set = set(rank_list)
rank_set

{'A', 'A+', 'B', 'B+', 'C', 'D'}

In [237]:
#복사본 만들기
df_ESG_copy = df_ESG.copy()

In [238]:
#mapping
rank_mapping = {
    'A+': 0,
    'A' : 1,
    'B+':2,
    'B':3,
    'C':4,
    'D':5
}

In [239]:
df_ESG.replace(rank_mapping, inplace=True)
df_ESG.head()

,기업명,종합등급_2021,환경_2021,사회_2021,지배구조_2021,종합등급_2020,환경_2020,사회_2020,지배구조_2020,종합등급_2019,환경_2019,사회_2019,지배구조_2019
0,AJ네트웍스,3,5,3,3,3,5,3,3,3,5,3,3
1,AK홀딩스,2,3,2,2,2,4,2,2,3,4,3,3
2,BGF,1,1,0,1,2,3,1,2,3,4,1,3
3,BGF리테일,1,1,0,1,1,2,2,1,3,4,2,2
4,BNK금융지주,0,1,0,0,0,2,0,0,0,2,0,0
